<a href="https://colab.research.google.com/github/nikhileshchapalge/loan-default-analysis/blob/main/Spark_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyarrow=="1.0.0"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.2 MB 698 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

In [5]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [8]:
from pyspark.sql.functions import *


In [9]:
import pyspark

In [10]:
pyspark.version


<module 'pyspark.version' from '/content/spark-3.3.1-bin-hadoop3/python/pyspark/version.py'>

In [14]:
## read the dataset
df=spark.read.option('header','true').csv('/content/census (2).csv',inferSchema=True)

In [11]:
spark

In [14]:
df

DataFrame[age: int, workclass: string, education: string, education_num: int, marital_status: string, occupation: string, relationship: string, race: string, gender: string, hours_per_week: int, native_country: string, greater_than_50k: int]

In [15]:
df.show()

+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+
|age|        workclass|    education|education_num|      marital_status|        occupation|  relationship|               race| gender|hours_per_week|native_country|greater_than_50k|
+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+
| 39|        State-gov|    Bachelors|           13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|            40| United-States|               0|
| 50| Self-emp-not-inc|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|            13| United-States|               0|
| 38|          Private|      HS-grad|            9|            Divorced| Handlers-cleaners

In [16]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- native_country: string (nullable = true)
 |-- greater_than_50k: integer (nullable = true)



In [15]:
df.columns

['age',
 'workclass',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'hours_per_week',
 'native_country',
 'greater_than_50k']

In [17]:
df.select(['workclass','education']).show()

+-----------------+-------------+
|        workclass|    education|
+-----------------+-------------+
|        State-gov|    Bachelors|
| Self-emp-not-inc|    Bachelors|
|          Private|      HS-grad|
|          Private|         11th|
|          Private|    Bachelors|
|          Private|      Masters|
|          Private|          9th|
| Self-emp-not-inc|      HS-grad|
|          Private|      Masters|
|          Private|    Bachelors|
|          Private| Some-college|
|        State-gov|    Bachelors|
|          Private|    Bachelors|
|          Private|   Assoc-acdm|
|          Private|    Assoc-voc|
|          Private|      7th-8th|
| Self-emp-not-inc|      HS-grad|
|          Private|      HS-grad|
|          Private|         11th|
| Self-emp-not-inc|      Masters|
+-----------------+-------------+
only showing top 20 rows



In [16]:
# to see the first row 
df.head(1)

[Row(age=39, workclass=' State-gov', education=' Bachelors', education_num=13, marital_status=' Never-married', occupation=' Adm-clerical', relationship=' Not-in-family', race=' White', gender=' Male', hours_per_week=40, native_country=' United-States', greater_than_50k=0)]

In [18]:
df.dtypes

[('age', 'int'),
 ('workclass', 'string'),
 ('education', 'string'),
 ('education_num', 'int'),
 ('marital_status', 'string'),
 ('occupation', 'string'),
 ('relationship', 'string'),
 ('race', 'string'),
 ('gender', 'string'),
 ('hours_per_week', 'int'),
 ('native_country', 'string'),
 ('greater_than_50k', 'int')]

In [19]:
df.describe().show()

+-------+------------------+------------+-------------+------------------+--------------+-----------------+------------+-------------------+-------+------------------+--------------+-------------------+
|summary|               age|   workclass|    education|     education_num|marital_status|       occupation|relationship|               race| gender|    hours_per_week|native_country|   greater_than_50k|
+-------+------------------+------------+-------------+------------------+--------------+-----------------+------------+-------------------+-------+------------------+--------------+-------------------+
|  count|             32560|       30724|        32560|             32560|         32560|            30717|       32560|              32560|  32560|             32560|         31977|              32560|
|   mean|38.581848894348894|        null|         null|10.080681818181818|          null|             null|        null|               null|   null|40.437469287469284|          null|0.2408

In [20]:
df.withColumnRenamed('native_country','country').show()

+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+
|age|        workclass|    education|education_num|      marital_status|        occupation|  relationship|               race| gender|hours_per_week|       country|greater_than_50k|
+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+
| 39|        State-gov|    Bachelors|           13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|            40| United-States|               0|
| 50| Self-emp-not-inc|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|            13| United-States|               0|
| 38|          Private|      HS-grad|            9|            Divorced| Handlers-cleaners

In [17]:
df.withColumn('age_in_months', df['age']*12).show()

+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+-------------+
|age|        workclass|    education|education_num|      marital_status|        occupation|  relationship|               race| gender|hours_per_week|native_country|greater_than_50k|age_in_months|
+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+-------------+
| 39|        State-gov|    Bachelors|           13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|            40| United-States|               0|          468|
| 50| Self-emp-not-inc|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|            13| United-States|               0|          600|
| 38|          Priva

In [21]:
df.show()

+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+
|age|        workclass|    education|education_num|      marital_status|        occupation|  relationship|               race| gender|hours_per_week|native_country|greater_than_50k|
+---+-----------------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+--------------+--------------+----------------+
| 39|        State-gov|    Bachelors|           13|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|            40| United-States|               0|
| 50| Self-emp-not-inc|    Bachelors|           13|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|            13| United-States|               0|
| 38|          Private|      HS-grad|            9|            Divorced| Handlers-cleaners

In [34]:
## read the dataset
df2=spark.read.option('header','true').csv('/content/marks data  - Sheet1 (1).csv',inferSchema=True)

In [35]:
df2.show()

+-----+----+----+----+
| Name| phy|chem|math|
+-----+----+----+----+
|  tom|  67|  50|  70|
|jerry|null|  89|  75|
|lance|  89|  67|null|
+-----+----+----+----+



In [36]:
df2.na.fill(70,['phy']).show()


+-----+---+----+----+
| Name|phy|chem|math|
+-----+---+----+----+
|  tom| 67|  50|  70|
|jerry| 70|  89|  75|
|lance| 89|  67|null|
+-----+---+----+----+



In [37]:
df2=df2.na.fill(70,['phy'])

In [38]:
df2.show()

+-----+---+----+----+
| Name|phy|chem|math|
+-----+---+----+----+
|  tom| 67|  50|  70|
|jerry| 70|  89|  75|
|lance| 89|  67|null|
+-----+---+----+----+



In [39]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

df_stats = df2.select(
    _mean(col('math')).alias('mean'),
    _stddev(col('math')).alias('std')
).collect()

mean = df_stats[0]['mean']
std = df_stats[0]['std']

In [42]:
mean

72.5